In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import random

# 数据读入

In [3]:
data = pd.read_csv("/Users/cys/Downloads/RandomForest/sonar-all-data.csv",header=None);
data.head(10)
train_data , test_data = train_test_split(data,test_size = 0.2 ,random_state = 46)

# 随机森林代码

In [10]:
def split(dataSet,feat,value):
    return dataSet[dataSet[feat] <= value],dataSet[dataSet[feat] > value]

def geni(dataSet):
    H = 1
    num = len(dataSet)
    for i in dataSet.iloc[:,-1].value_counts():
        p = i/num
        H = H -  p*p
    return H

def getClass(dataSet):
    return dataSet.iloc[:,-1].value_counts().index[0]
    
def split_best(dataSet,featList,C=2):
    num = len(dataSet)
    if(num <= C):
        return None,getClass(dataSet)#结点最小
    if(len(set(dataSet.iloc[:,-1].values)) == 1):
        return None,getClass(dataSet)
    
    
    best_feat = - 1
    best_H = math.inf
    best_value = -1
    
    for feat in featList:
        value_list = set(dataSet[feat].values)
        for value in value_list:
            data_1,data_2 = split(dataSet,feat,value)
            num1 = len(data_1)
            num2 = len(data_2)
            if(num1 < 1 or num2 < 1):
                continue
            H = num1/num*geni(data_1)+num2/num*geni(data_2)
            if(H < best_H):
                best_H = H
                best_feat = feat
                best_value = value
    return best_feat,best_value


def CART(dataSet,featList,C=2):
    feat,value = split_best(dataSet,featList,C)
    if(feat == None):
        return value
    tree = {}
    tree['feat'] = feat
    tree['value'] = value
    data_1,data_2 = split(dataSet,feat,value)
    tree['left'] = CART(data_1,featList,C)
    tree['right'] = CART(data_2,featList,C)
    return tree

def predict(tree,data):
    T = tree
    while(type(T) == dict):
        if(data.iloc[0,T['feat']] <= T['value']):
            T = T['left']
        else:
            T = T['right']
    return T

def bootstrap(dataSet,N):
    num = len(dataSet)
    l = []
    for i in range(N):
        l.append(random.randrange(0,num))
    return dataSet.iloc[l]

def randomfeatList(featList,M):
    num = len(featList)
    f = []
    for i in range(M):
        f.append(random.randrange(0,num))
    return f

def RF(dataSet,N=10,M=10,C=2,treeNum=10):
    num = len(dataSet)
    f_num = len(list(dataSet))
    f_list = [x for x in range(f_num-1)]
    Rtree = []
    for i in range(treeNum):
        featureList = randomfeatList(f_list,M)
        dataSet_sub = bootstrap(dataSet,N)
        tree = CART(dataSet_sub,featureList,C)
        print(i)
        Rtree.append(tree)
    return Rtree

def RFpredict(Rtree,data):
    l = []
    for tree in Rtree:
        l.append(predict(tree,data))
    base_count = 0
    for i in l:
        if(l.count(i) > base_count):
            base_count = l.count(i)
            ans = i
    return ans
    

def accuracy(dataSet,Rtree):
    num = len(dataSet)
    count = 0;
    for i in range(num):
        ans = RFpredict(Rtree,dataSet.iloc[[i]])
        if(ans == dataSet.iloc[i,-1]):
            count = count + 1
    print(count/num)

In [11]:
Rtree = RF(train_data,208,10,2,129)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128


In [12]:
accuracy(test_data,Rtree)

0.8571428571428571


# sklearn 实现

In [5]:
RandomForestClassifier(n_estimators=9000,max_features=10)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=9000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [7]:
rf = RandomForestClassifier(n_estimators=9000,max_features=10)
rf.fit(train_data.iloc[:,:-1],train_data.iloc[:,-1])
count = 0
num = len(test_data)
i = 0
for ans in rf.predict(test_data.iloc[:,:-1]):
    if(ans == test_data.iloc[i,-1]):
        count = count + 1
    i = i + 1
print(count/num)

0.9523809523809523


# **permutation test:**   
&nbsp;&nbsp;&nbsp;&nbsp;~~importance(i) = performance(D) - performance(D^p)~~   
&nbsp;&nbsp;&nbsp;&nbsp;importance(i) = Eoob(G) - Eoob(G^p)   
&nbsp;&nbsp;&nbsp;&nbsp;importance(i) = Eoob(G) - Eoob^p(G)   
&nbsp;&nbsp;&nbsp;&nbsp; *为了特征选择*

## 还可以作出类似large-margin结果

In [17]:
data[1].value_counts().keys()

Float64Index([0.0509, 0.0378, 0.0215, 0.0186, 0.0279, 0.0421, 0.0308, 0.0213,
              0.0453, 0.0222,
              ...
              0.0216, 0.0104, 0.0353, 0.0627, 0.0019, 0.0375, 0.0321, 0.0808,
              0.0281, 0.0902],
             dtype='float64', length=182)